# 뉴스 빅데이터 분석 데이터 추출 & 사용자 사전 추가

In [82]:
import pandas as pd 
from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Mecab, Twitter, Kkma
from konlpy.utils import pprint
def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
stopwords = Stopwords('./data/koreanStopwords.txt') +Stopwords('./data/newspress.txt')

In [ ]:
def Extract_Keyword_from_news_bigdata(path):
    from glob import glob
    fileList = glob(path+'*.csv')
    listIs =list()
    for i in fileList:
        df = pd.read_csv(i,engine='python')
        f = list(filter(lambda x: x[1] in ['중요키워드','키워드','keword','토픽키워드','토픽 키워드'], enumerate(df.columns.values)))
        if not len(f) == 0:
            listIs += list(df[df.columns[f[0][0]]].values)
    outlist = []
    for ix in list(filter(lambda x: type(x)!=float, listIs)):
        outlist += ix.split(',')
    outlist1 = list(map(lambda x: ''.join(x.split('_')), outlist))
    outlist2 = list(map(lambda x: ' '.join(x.split('_')), outlist))
    return outlist1+outlist2

In [ ]:
dbpath = '/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ckonlpy/data/twitter/noun/'

In [ ]:
path1 = './data/news/high_frequency_noun/'
path2 = './data/news/mainissue/'
path3 = './data/news/newstopic/'
path4 = './data/news/people/'
path5 = './data/news/4th_industry/'
path6 = './data/news/have_negative_positive/constitution/'
path7 = './data/news/have_negative_positive/household_debt/'
path8 = './data/news/have_negative_positive/olymphic/'
path9 = './data/news/have_negative_positive/'
path10 = './data/news/'

pathlist = [path1, path2, path3,
            path4, path5, path6,
            path7, path8, path9,
           path10]

with open(dbpath+'from_news.txt','w') as f:
    for path in pathlist:
        out = Extract_Keyword_from_news_bigdata(path)
        f.write('\n'.join(out))

In [83]:
ct = ctwitter()
ot = Twitter()
xxxx = ct._dictionary._pos2words
list(map(lambda x: len(xxxx[x]), xxxx))

[512, 5068471, 5532]

In [84]:
mecab = Mecab()
kkma = Kkma()

# 자연어 처리

# 분석

In [160]:
# -*- coding: utf-8 -*-
import sys
sys.path.append('/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/')
from collections import defaultdict
from datetime import datetime, timedelta
import time, re, pickle, itertools
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
import urllib3, json
import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize
import numpy as np
from operator import itemgetter
import os

def OpenAPI():
    import pickle
    openapi = pickle.load(open('./etri_open_api_access_key', 'rb'))
    return openapi
def USE_ETRI_ANALYSIS(analysisCode, text):    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    accessKey = OpenAPI().strip()
    requestJson = {
        'access_key' : accessKey,
        'argument' : {
            'text' : text,
            'analysis_code' : analysisCode
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
    return json.loads(str(response.data,'utf-8')), str(response.status)

def USE_ETRI_INFO(fWord, sWord):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/WordRel"
    accessKey = OpenAPI().strip()
    firstWord = fWord
    #firstSenseId = fSenseId
    secondWord = sWord
    #secondSenseId = sSenseId
    requestJson = {
        "access_key": accessKey,
        "argument": {
        'first_word': firstWord,
        #'first_sense_id': firstSenseId,
        'second_word': secondWord,
        #'second_sense_id': secondSenseId
        }}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson))
    return json.loads(str(response.data, 'utf-8')), str(response.status)

def USE_ETRI_RELATION(word):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/Word"
    accessKey = OpenAPI().strip()
    word = word
    requestJson = {
    "access_key": accessKey,
    "argument": {"word": word}}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},
                            body=json.dumps(requestJson))
    return json.loads(str(response.data, 'utf-8')), str(response.status)

def Extract_Text_Info(idx):
    morp = list(map(lambda x: (x['lemma'], x['type']),idx['morp']))
    wsd = list(map(lambda x: (x['text'], x['type']),idx['WSD']))
    word = list(map(lambda x: (x['text'], x['type']),idx['word']))
    ne = list(map(lambda x: (x['text'], x['type']),idx['NE']))
    return morp, wsd, word, ne

def Run_ETRI_Analysis(stopwordsList, text):
    etri = USE_ETRI_ANALYSIS('srl', text)['return_object']    
    etri = etri['sentence']
    morp = [] ; wsd = [] ; word = [] ; ne = [] ; srl = [] ; dependency = []
    for i in etri:
        x = Extract_Text_Info(i)
        morp += x[0]
        wsd += x[1]
        word +=x[2]
        ne += x[3]
        for ii in i['SRL']:
            srl += list(map(lambda x: (x['text'], x['type']),ii['argument']))
        dependency += list(map(lambda x: (x['text'], x['label']), i['dependency']))
    return morp, wsd, word, ne, srl, dependency, etri
    
if __name__ == '__main__':
    pass

In [207]:
site = 'Naver'
#site = 'daum'
if site == 'daum':
    collection = 'newsDaum'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/daum/'
elif site.lower() == 'naver':
    collection = 'newsNaver'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/naver/'

In [209]:
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, collection)

In [210]:
keywordList = list()
pressList = list()
#etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/'
for data in useCollection.find({'site':site}):
    pressList.append(data['press'])
    if len(data['mainText'].strip()) !=0:
        if site=='daum':
            if data['keywords'] !='NaN':
                keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
        else:
            pass
        idIs = data['_id']._ObjectId__id
        if not os.path.isfile(etri_outcome+idIs.hex()+'.picked.txt'):
            print (idIs.hex())
            try:
                etri = USE_ETRI_ANALYSIS('srl', re.sub('\. ','.\n', data['mainText']))
            except:
                print (etri[1])
                break
            else:
                pickle.dump(etri[0], open(etri_outcome+idIs.hex()+'.picked.txt','wb'))

5a303059588c135900450bd9
5a303059588c135900450bd1
5a303059588c135900450bc9
5a303059588c135900450bc7
5a303059588c135900450bc6
5a303059588c135900450bc5
5a303059588c135900450bc4
5a303059588c135900450bc3
5a303059588c135900450bbe
5a303059588c135900450bbc
5a303059588c135900450bbb
5a303059588c135900450bb9
5a303059588c135900450bb6
5a303059588c135900450bb2
5a303059588c135900450bb1
5a303059588c135900450bb0
5a303059588c135900450bac
5a303059588c135900450bbf
5a303059588c135900450bab
5a303059588c135900450ba9
5a303059588c135900450ba8
5a303059588c135900450ba6
5a303059588c135900450ba3
5a303059588c135900450ba2
5a303059588c135900450b9e
5a303059588c135900450b9c
5a303059588c135900450b99
5a303059588c135900450b97
5a303059588c135900450b95
5a303059588c135900450b8f
5a303059588c135900450b8e
5a303059588c135900450b89
5a303059588c135900450b87
5a303059588c135900450b86
5a303059588c135900450b84
5a303059588c135900450b7f
5a303059588c135900450b7d
5a303059588c135900450b7c
5a303059588c135900450b7b
5a303059588c135900450b79


5a2d8322588c1358bcf3ed3e
5a2d8322588c1358bcf3ed3d
5a2d8322588c1358bcf3ed3b
5a2d8322588c1358bcf3ed3a
5a2d8322588c1358bcf3ed39
5a2d8322588c1358bcf3ed37
5a303059588c135900450bad
5a2d8322588c1358bcf3ed36
5a2d8322588c1358bcf3ed34
5a2d8322588c1358bcf3ed33
5a2d8322588c1358bcf3ed31
5a2d8322588c1358bcf3ed29
5a2d8322588c1358bcf3ed26
5a2d8322588c1358bcf3ed25
5a2d8322588c1358bcf3ed23
5a2d8322588c1358bcf3ed22
5a2d8322588c1358bcf3ed21
5a2d8322588c1358bcf3ed20
5a2d8322588c1358bcf3ed1f
5a2d8322588c1358bcf3ed1e
5a2d8322588c1358bcf3ed1d
5a2d8322588c1358bcf3ed1c
5a2d8322588c1358bcf3ed1a
5a2d8322588c1358bcf3ed19
5a2d8322588c1358bcf3ed17
5a2d8322588c1358bcf3ed15
5a2d8322588c1358bcf3ed13
5a2d8322588c1358bcf3ed10
5a303059588c135900450b3f
5a2d8322588c1358bcf3ed0f
5a2d8322588c1358bcf3ed0b
5a2d8322588c1358bcf3ed0a
5a2d8322588c1358bcf3ed09
5a2d8322588c1358bcf3ed08
5a2d8322588c1358bcf3ed07
5a2d8322588c1358bcf3ed06
5a2d8322588c1358bcf3ed03
5a2d8322588c1358bcf3ecff
5a2d8322588c1358bcf3ecfc
5a2d8322588c1358bcf3ecfa


5a2b8ac6588c1357cce1ff34
5a2b8ac6588c1357cce1ff33
5a2b8ac6588c1357cce1ff32
5a2b8ac6588c1357cce1ff2e
5a2b8ac6588c1357cce1ff2d
5a303059588c135900450bc8
5a2b8ac6588c1357cce1ff2c
5a2b8ac6588c1357cce1ff2b
5a2b8ac6588c1357cce1ff29
5a2b8ac6588c1357cce1ff28
5a2b8ac6588c1357cce1ff26
5a2b8ac6588c1357cce1ff25
5a2d8322588c1358bcf3ed24
5a2b8ac6588c1357cce1ff24
5a2d8322588c1358bcf3ede2
5a2c53cd588c1304acddf784
5a2b8ac6588c1357cce1ff22
5a2b8ac6588c1357cce1ff1f
5a2b8ac6588c1357cce1ff1e
5a2b8ac6588c1357cce1ff1d
5a2b8ac6588c1357cce1ff1c
5a2b8ac6588c1357cce1ff1b
5a2b8ac6588c1357cce1ff18
5a2b8ac6588c1357cce1ff17
5a2b8ac6588c1357cce1ff16
5a2b8ac6588c1357cce1ff76
5a2b8ac6588c1357cce1ff15
5a2b8ac6588c1357cce1ff12
5a2b8ac6588c1357cce1ff0f
5a303059588c135900450b2a
5a2b8ac6588c1357cce1ff0e
5a2b8ac6588c1357cce1ff0d
5a2b8ac6588c1357cce1ff0c
5a2ec677588c1360805f8a0a
5a2d8322588c1358bcf3ed56
5a2b8ac6588c1357cce1ff9e
5a2a3ae1588c134d58538edf
5a2c53cd588c1304acddf74f
5a2a3ae1588c134d58538ede
5a29d24d588c1367a04243f5


5a29d494588c13371415ac14
5a29d494588c13371415ac88
5a29c445588c132954d19802
5a29d494588c13371415ac13
5a29d494588c13371415ac11
5a29d494588c13371415ac77
5a2a404c588c131d18abe3ed
5a29d494588c13371415aca3
5a303059588c135900450b43
5a2b8ac6588c1357cce1fede
5a29d494588c13371415ac0d
5a2a3ae1588c134d58538f63
5a2a4085588c136d007f0ce6
5a2a3ae1588c134d58538eb0
5a2a3ae1588c134d58538eef
5a2c53cd588c1304acddf78c
5a2b8ac6588c1357cce1feda
5a2a404c588c131d18abe390
5a29d494588c13371415ac0b
5a29c445588c132954d19819
5a29d494588c13371415ac0a
5a2a3ae1588c134d58538ed5
5a2d8322588c1358bcf3edd1
5a29d494588c13371415ac09
5a2ec677588c1360805f8a53
5a2b8ac6588c1357cce1ff2a
5a2a452f588c1363b00902c8
5a29d494588c13371415ac08
5a29d494588c13371415ac07
5a29d24d588c1367a04243da
5a29d494588c13371415ac05
5a2a4085588c136d007f0c58
5a2b8ac6588c1357cce1fef1
5a29d24d588c1367a04243a7
5a29d24d588c1367a0424466
5a29d24d588c1367a0424464
5a303059588c135900450bcf
5a303059588c135900450ba5
5a2ec677588c1360805f8ac3
5a2a3ae1588c134d58538e81


5a29c445588c132954d197c5
5a29c445588c132954d197b6
5a29c445588c132954d197ef
5a29c445588c132954d197a2
5a2a404c588c131d18abe3fa
5a29c445588c132954d1975e
5a2b8ac6588c1357cce1fed1
5a29d24d588c1367a042444e
5a2d8322588c1358bcf3ed45
5a29d494588c13371415ace1
5a29d24d588c1367a04243b5
5a2d8322588c1358bcf3edb6
5a29d24d588c1367a04243e8
5a29d494588c13371415ac9d
5a29c445588c132954d197fe
5a29d494588c13371415acad
5a29d24d588c1367a0424465
5a29c445588c132954d1979e
5a2a3ae1588c134d58538e8c
5a2a4085588c136d007f0ce4
5a29c445588c132954d197a8
5a2a452f588c1363b009028f
5a29c445588c132954d1979a
5a29d24d588c1367a042439d
5a2b8ac6588c1357cce1fef2
5a29d24d588c1367a04243a0
5a2ec677588c1360805f8aa0
5a2a404c588c131d18abe3eb
5a303059588c135900450b27
5a2a3ae1588c134d58538ed0
5a29d494588c13371415ac90
5a29c445588c132954d197de
5a2a452f588c1363b0090261
5a2a452f588c1363b0090255
5a29c445588c132954d197c4
5a29d24d588c1367a042443a
5a29c445588c132954d19763
5a2a3ae1588c134d58538f30
5a29d494588c13371415ac25
5a29c445588c132954d1980a


...

5a2a3ae1588c134d58538ee4
5a2a3ae1588c134d58538ee6
5a2a3ae1588c134d58538f10
5a29d494588c13371415accb
5a2a3ae1588c134d58538ee8
5a2ec677588c1360805f8ace
5a2a3ae1588c134d58538ee9
5a2d8322588c1358bcf3ed86
5a2a4085588c136d007f0c57
5a303059588c135900450bd7
5a2a3ae1588c134d58538eeb
5a29c445588c132954d1981c
5a29c445588c132954d19777
5a2a3ae1588c134d58538ef0
5a2a3ae1588c134d58538ef2
5a2b8ac6588c1357cce1ff83
5a2a3ae1588c134d58538ef6
5a303059588c135900450b77
5a2a452f588c1363b00902d4
5a29d494588c13371415ac2e
5a2a3ae1588c134d58538ef7
5a2b8ac6588c1357cce1ff11
5a2a3ae1588c134d58538ef8
5a2b8ac6588c1357cce1ff78
5a2a3ae1588c134d58538ef9
5a29c445588c132954d1977c
5a2b8ac6588c1357cce1feef
5a2d8322588c1358bcf3ed28
5a2a3ae1588c134d58538efc
5a29d494588c13371415ac94
5a2a3ae1588c134d58538efd
5a29c445588c132954d19820
5a2a452f588c1363b00902e7
5a2a3ae1588c134d58538eff
5a2a3ae1588c134d58538f03
5a29d494588c13371415ac81
5a2a3ae1588c134d58538f07
5a303059588c135900450b62
5a2a3ae1588c134d58538f08
5a2a3ae1588c134d58538ed1


5a2a404c588c131d18abe3fc
5a2a404c588c131d18abe3fe
5a2c53cd588c1304acddf6fc
5a2a404c588c131d18abe400
5a2a452f588c1363b009021f
5a2a404c588c131d18abe401
5a303059588c135900450b55
5a29d24d588c1367a0424424
5a2a404c588c131d18abe404
5a2a404c588c131d18abe405
5a2a452f588c1363b00902a5
5a2a404c588c131d18abe406
5a2a404c588c131d18abe409
5a2a4085588c136d007f0cc6
5a29c445588c132954d197af
5a2a404c588c131d18abe40a
5a2ec677588c1360805f8a07
5a2a404c588c131d18abe40c
5a2a404c588c131d18abe40e
5a303059588c135900450afe
5a2a404c588c131d18abe40f
5a2a404c588c131d18abe419
5a2a404c588c131d18abe3a3
5a2a4085588c136d007f0c7b
5a2a4085588c136d007f0c48
5a2a404c588c131d18abe412
5a2a404c588c131d18abe413
5a29d24d588c1367a0424435
5a2a404c588c131d18abe414
5a2a404c588c131d18abe415
5a2d8322588c1358bcf3ed53
5a2a404c588c131d18abe416
5a2a4085588c136d007f0c7d
5a2a4085588c136d007f0c33
5a2a404c588c131d18abe418
5a29d24d588c1367a0424461
5a2a4085588c136d007f0cb6
5a303059588c135900450b98
5a2a452f588c1363b009030b
5a2a4085588c136d007f0d01


5a2a4085588c136d007f0ce3
5a29d24d588c1367a04243a9
5a2a452f588c1363b00902d5
5a2a4085588c136d007f0ce7
5a2d8322588c1358bcf3edd2
5a2a4085588c136d007f0ce8
5a2a4085588c136d007f0ce9
5a2c53cd588c1304acddf76e
5a2a4085588c136d007f0cef
5a29c445588c132954d197d2
5a2a4085588c136d007f0cf0
5a2a4085588c136d007f0cf3
5a2a4085588c136d007f0cf5
5a303059588c135900450b4a
5a303059588c135900450b29
5a2d8322588c1358bcf3ed62
5a2a4085588c136d007f0cf6
5a2a452f588c1363b009023b
5a2a4085588c136d007f0cf8
5a2a4085588c136d007f0cfe
5a2a4085588c136d007f0c2c
5a2a4085588c136d007f0d04
5a2a452f588c1363b0090260
5a2a4085588c136d007f0d05
5a2a404c588c131d18abe3f5
5a2a404c588c131d18abe389
5a2a4085588c136d007f0d07
5a2a3ae1588c134d58538f46
5a2a4085588c136d007f0d09
5a2a4085588c136d007f0d0a
5a2a3ae1588c134d58538eac
5a2a4085588c136d007f0d0d
5a2c53cd588c1304acddf707
5a29c445588c132954d19827
5a2a4085588c136d007f0d0f
5a29d494588c13371415acee
5a2a4085588c136d007f0d10
5a29d494588c13371415ac92
5a2a4085588c136d007f0d13
5a2b8ac6588c1357cce1ff6c


5a31691f588c1331140fad28
5a31691f588c1331140fad29
5a31691f588c1331140fad2a
5a31691f588c1331140fad2b
5a31691f588c1331140fad2c
5a31691f588c1331140fad2d
5a31691f588c1331140fad2e
5a31691f588c1331140fad2f
5a31691f588c1331140fad30
5a31691f588c1331140fad31
5a31691f588c1331140fad32
5a31691f588c1331140fad33
5a31691f588c1331140fad34
5a31691f588c1331140fad35
5a31691f588c1331140fad36
5a31691f588c1331140fad37
5a31691f588c1331140fad38
5a31691f588c1331140fad39
5a31691f588c1331140fad3a
5a31691f588c1331140fad3b
5a31691f588c1331140fad3c
5a31691f588c1331140fad3d
5a31691f588c1331140fad3e
5a31691f588c1331140fad3f
5a31691f588c1331140fad40
5a31691f588c1331140fad41
5a31691f588c1331140fad42
5a31691f588c1331140fad43
5a31691f588c1331140fad44
5a31691f588c1331140fad45
5a31691f588c1331140fad46
5a31691f588c1331140fad47
5a31691f588c1331140fad48
5a31691f588c1331140fad49
5a31691f588c1331140fad4a
5a31691f588c1331140fad4b
5a31691f588c1331140fad4c
5a31691f588c1331140fad4d
5a31691f588c1331140fad4e
5a31691f588c1331140fad4f


5a32e268588c136a189e66cc
5a32e268588c136a189e66cd
5a32e268588c136a189e66ce
5a32e268588c136a189e66cf
5a32e268588c136a189e66d0
5a32e268588c136a189e66d1
5a32e268588c136a189e66d2
5a32e268588c136a189e66d3
5a32e268588c136a189e66d4
5a32e268588c136a189e66d5
5a32e268588c136a189e66d6
5a32e268588c136a189e66d7
5a32e268588c136a189e66d8
5a32e268588c136a189e66d9
5a32e268588c136a189e66da
5a32e268588c136a189e66db
5a32e268588c136a189e66dc
5a32e268588c136a189e66dd
5a32e268588c136a189e66de
5a32e268588c136a189e66df
5a32e268588c136a189e66e0
5a32e268588c136a189e66e1
5a32e268588c136a189e66e2
5a32e268588c136a189e66e3
5a32e268588c136a189e66e4
5a32e268588c136a189e66e5
5a32e268588c136a189e66e6
5a32e268588c136a189e66e7
5a32e268588c136a189e66e8
5a32e268588c136a189e66e9
5a32e268588c136a189e66ea
5a32e268588c136a189e66eb
5a32e268588c136a189e66ec
5a32e268588c136a189e66ed
5a32e268588c136a189e66ee
5a32e268588c136a189e66ef
5a32e268588c136a189e66f0
5a32e268588c136a189e66f1
5a32e268588c136a189e66f2
5a32e268588c136a189e66f3


In [182]:
rawdata['_id']._ObjectId__id.hex() == a.hex()

True

In [167]:
bytes..fromhex(rawdata['_id']._ObjectId__id.hex())

b'Z*\xf4\x15X\x8c\x13v\xf0\xd6z\xfe'

In [59]:
rawdata = useCollection.find_one({'category':"뉴스"})
#rawdata

In [60]:
useCollection2 = dh.Use_Collection(useDb, 'newsNaver')

In [61]:
rawdata2 = useCollection2.find_one({'category':"정치"})
#rawdata2

In [62]:
from krwordrank.word import KRWordRank
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer, RegexTokenizer
from soynlp.noun import LRNounExtractor
from newspaper import Article
class Sentences:
    def __init__(self, text):
        self.text = text
        self.length = 0
    def __iter__(self):
            for doc in kkma.sentences(self.text):
                doc = doc.strip()
                if not doc:
                    continue
                for sent in doc.split(' '):
                    yield sent
    def __len__(self):
        if self.length == 0:
            for doc in kkma.sentences(self.text):
                doc = doc.strip()
                if not doc:
                    continue
                self.length += len(doc.split(' '))
        return self.length
def RunLRNounExtractor(text, min_count):
    verbose=False ; left_max_length=10 ; right_max_length=5
    #min_count=2
    word_extractor = WordExtractor()
    noun_extractor = LRNounExtractor(None, verbose, left_max_length, 
                                 right_max_length, min_count, 
                                 word_extractor)
    sentences = Sentences(text)
    nouns = noun_extractor.train_extract(sentences, minimum_noun_score=0.85, min_count=min_count)
    return nouns

def RunWordExtractor(text, min_count):
    word_extractor = WordExtractor(min_count=min_count,
                               min_cohesion_forward=0.7, 
                               min_right_branching_entropy=0.1)
    sentences = Sentences(text)
    word_extractor.train(sentences)
    words = word_extractor.extract()
    return words

def RunWordKRwordRank(text, min_count):
    from krwordrank.hangle import normalize
    #min_count = 2   # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10 # 단어의 최대 길이
    wordrank_extractor = KRWordRank(min_count, max_length)
    beta = 0.85   # PageRank의 decaying factor beta
    max_iter = 300
    verbose = False
    textIs = text.split('. ')
    textIs = list(map(lambda x: normalize(x, english = True, number = True), textIs))
    keywords, rank, graph = wordrank_extractor.extract(textIs, beta, max_iter, verbose)
    return keywords, rank, graph

In [63]:
def Extract_Keywords2(text, stopwords):
    vect = TfidfVectorizer(norm='l2', 
                           ngram_range=(1,4),
                           use_idf = True,sublinear_tf = True, 
                           stop_words=stopwords)
    vect.fit(list(map(lambda x: x[0], ct.pos(text))))
    #vect.fit(ct.phrases(text))
    #vect.fit(ct.nouns(text))
    #vect.fit(ct.morphs(text))
    #vect.fit(list(map(lambda x: x[0], ot.pos(text,stem = True, norm = True))))
    count = vect.transform(kkma.sentences(text)).toarray().sum(axis = 0)
    idx = np.argsort(-count)
    count = count[idx]
    feature_name = np.array(vect.get_feature_names())[idx]
    out = dict(zip(feature_name, count))
    return sorted(out.items(), key = itemgetter(1), reverse=True)

def CombinationAndCheckInclude(dfidfDict):
    outset = set()
    for x1, x2 in list(itertools.combinations(list(dfidfDict.keys()), 2)):
        if x1 in x2:
            if dfidfDict[x1] <= dfidfDict[x2]:
                outset.add(x1)
            else:
                outset.add(x2)
        elif x2 in x1:
            if dfidfDict[x1] >= dfidfDict[x2]:
                outset.add(x2)
            else:
                outset.add(x1)
        else:
            pass
    return outset

def Out_Keywords(text, stopwords):
    dfidfOut = dict(Extract_Keywords2(text, stopwords))
    forFilterSet = CombinationAndCheckInclude(dfidfOut)
    reout = list(filter(lambda x: not x in forFilterSet, dfidfOut.keys()))
    return reout

In [ ]:
data['mainText']

In [ ]:
vect1 = CountVectorizer(tokenizer=ct.phrases, ngram_range=(1,1),
                        stop_words=stopwords)
count1 = vect1.fit_transform(re.sub('\. ','.\n', data['mainText']).split('\n')).toarray().sum(axis = 0)
idx1 = np.argsort(-count1)
count1 = count1[idx1]
feature_name1 = np.array(vect1.get_feature_names())[idx1]
out1 = dict(zip(feature_name1, count1))
sorted(out1.items(), key=lambda x: x[1], reverse=True)

In [ ]:
vect3 = CountVectorizer(tokenizer=ct.nouns, ngram_range=(1,2),
                    stop_words=stopwords)
count3 = vect3.fit_transform(re.sub('\. ','.\n', data['mainText']).split('\n')).toarray().sum(axis = 0)
idx3 = np.argsort(-count3)
count3 = count3[idx3]
feature_name3 = np.array(vect3.get_feature_names())[idx3]
out3 = dict(zip(feature_name3, count3))
sorted(out3.items(), key=lambda x: x[1], reverse=True)

In [ ]:
vect4 = CountVectorizer(tokenizer=ct.morphs, ngram_range=(1,3),
                    stop_words=stopwords)
count4 = vect4.fit_transform(re.sub('\. ','.\n', data['mainText']).split('\n')).toarray().sum(axis = 0)
idx4 = np.argsort(-count4)
count4 = count4[idx4]
feature_name4 = np.array(vect4.get_feature_names())[idx4]
out4 = dict(zip(feature_name4, count4))
sorted(out4.items(), key=lambda x: x[1], reverse=True)

In [ ]:
pos_tagger = Twitter()
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [ ]:
train_docs = [(tokenize(row[1].decode('utf-8')), row[2]) for row in train_data[:3000]]

In [ ]:
from gensim.models.doc2vec import TaggedDocument

In [ ]:
def Extract_Keyword_from_news_bigdata(path):
    from glob import glob
    fileList = glob(path+'*.csv')
    listIs =list()
    for i in fileList:
        df = pd.read_csv(i,engine='python')
        f = list(filter(lambda x: x[1] in ['중요키워드','키워드','keword','토픽키워드','토픽 키워드'], enumerate(df.columns.values)))
        if not len(f) == 0:
            listIs += list(df[df.columns[f[0][0]]].values)
    outlist = []
    for ix in list(filter(lambda x: type(x)!=float, listIs)):
        outlist += ix.split(',')
    outlist1 = list(map(lambda x: ''.join(x.split('_')), outlist))
    outlist2 = list(map(lambda x: ' '.join(x.split('_')), outlist))
    return outlist1+outlist2

In [ ]:
from gensim.models import doc2vec

In [2]:
from glob import glob

In [3]:
def Extract_Maintext_from_news_bigdata(path):
    from glob import glob
    fileList = glob(path+'*.csv')
    listIs =list()
    for i in fileList:
        df = pd.read_csv(i,engine='python')
        f = list(filter(lambda x: x[1] in ['본문'], enumerate(df.columns.values)))
        if not len(f) == 0:
            listIs += list(df[df.columns[f[0][0]]].values)
    outlist = []
    for ix in list(filter(lambda x: type(x)!=float, listIs)):
        outlist += ix.split(',')
    outlist1 = list(map(lambda x: ''.join(x.split('_')), outlist))
    outlist2 = list(map(lambda x: ' '.join(x.split('_')), outlist))
    return outlist1+outlist2

In [46]:
path1 = './data/news/high_frequency_noun/'
path2 = './data/news/mainissue/'
path3 = './data/news/newstopic/'
path4 = './data/news/people/'
path5 = './data/news/4th_industry/'
path6 = './data/news/have_negative_positive/constitution/'
path7 = './data/news/have_negative_positive/household_debt/'
path8 = './data/news/have_negative_positive/olymphic/'
path9 = './data/news/have_negative_positive/'
path10 = './data/news/'
path11 = './data/news/speaking/'
path12 = './data/news/quote/'

pathlist = [path1, path2, path3,
            path4, path5, path6,
            path7, path8, path9,
           path10, path11, path12]

In [48]:
for i in pathlist:
    fileList = glob(i+'*.csv')
    fileList.extend(glob(i+'*.CSV'))
    for ii in fileList:
        df = pd.read_csv(ii, encoding='utf-8')
        if '본문' in list(df.columns.values):
            dff = df.copy()
            print (dff.head())
            f = list(filter(lambda x: x[1] in ['본문'], enumerate(dff.columns.values)))
            print (f)
            break
            

   Unnamed: 0                                                 주소        일자  \
0           0  http://www.bigkinds.or.kr/news/newsDetailView....  20170701   
1           1  http://www.bigkinds.or.kr/news/newsDetailView....  20170701   
2           2  http://www.bigkinds.or.kr/news/newsDetailView....  20170701   
3           3  http://www.bigkinds.or.kr/news/newsDetailView....  20170701   
4           4  http://www.bigkinds.or.kr/news/newsDetailView....  20170701   

     언론사     기고자                                        제목   통합 분류1  \
0   서울경제  고병기 기자    [부동산시장 뒤집어보기]같은 전략, 다른 결과에 웃다가 운 경매 시장   경제>부동산   
1  헤럴드경제  김유진 기자               트럼프 인도 부동산, 모디 총리 세제 개편에 타격  경제>국제경제   
2   중도일보  임붕순 기자               서산준법지원센터, 서산평생학습센터 시민로스쿨 실시       문화   
3   매일경제     NaN  현대산업개발, 초역세권 아파트 ‘인덕 아이파크’ 견본주택에 실수요자 북적   경제>부동산   
4  헤럴드경제  김우영 기자                     새로 생긴 길따라 아파트 단지 살펴볼까      NaN   

          통합 분류2  통합 분류3 사건/사고 분류1 사건/사고 분류2 사건/사고 분류3  \
0      경제>금융_재테크     NaN       NaN       NaN  

In [ ]:
keywordList = []
press = []
ii = 0
for data in useCollection.find({'site':'daum'}):
    if len(data['mainText'].strip()) !=0:
        ii +=1
        if data['keywords'] !='NaN':
            keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
        press.append(data['press'])
        f = Out_Keywords(data['mainText'],stopwords)
        print (f, data['keywords'])
        if ii== 50:break
        break

In [ ]:
re.sub('\. ','.\n', data['mainText']).split('\n')

In [105]:
etri = Run_ETRI_Analysis(stopwords, re.sub('\. ','.\n', rawdata['mainText']))

In [115]:
x1 = list(map(lambda x: x[0], etri[0]))
x2 = list(map(lambda x: x[0], etri[1]))
x3 = list(map(lambda x: x[0], etri[2]))
x4 = list(map(lambda x: x[0], etri[3]))
x5 = list(map(lambda x: x[0], etri[4]))
x6 = list(map(lambda x: x[0], etri[5]))

In [117]:
etri[6]

[{'NE': [{'begin': 8.0,
    'common_noun': 0.0,
    'end': 8.0,
    'id': 0.0,
    'text': '밥',
    'type': 'CV_FOOD',
    'weight': 0.436106}],
  'SRL': [{'argument': [{'text': '사람들',
      'type': 'ARGM-CND',
      'weight': 0.109902,
      'word_id': 0.0},
     {'text': '식사', 'type': 'ARG1', 'weight': 0.270891, 'word_id': 1.0}],
    'sense': 1.0,
    'verb': '하',
    'weight': 0.190396,
    'word_id': 2.0},
   {'argument': [{'text': '밥과 반찬',
      'type': 'ARG1',
      'weight': 0.410294,
      'word_id': 5.0}],
    'sense': 1.0,
    'verb': '먹',
    'weight': 0.410294,
    'word_id': 6.0},
   {'argument': [{'text': '사람들',
      'type': 'ARGM-CND',
      'weight': 0.169718,
      'word_id': 0.0},
     {'text': '때', 'type': 'ARGM-CND', 'weight': 0.275581, 'word_id': 3.0},
     {'text': '순서', 'type': 'ARG1', 'weight': 0.486648, 'word_id': 7.0}],
    'sense': 1.0,
    'verb': '다르',
    'weight': 0.310649,
    'word_id': 8.0}],
  'WSD': [{'begin': 0.0,
    'end': 0.0,
    'id': 0.0,
   

In [155]:
import os

In [158]:
import os
if not os.path.isfile(etri_outcome+rawdata['_id']._ObjectId__id.hex()+'.picked.txt'):
    pickle.dump(etri[6], open(etri_outcome+rawdata['_id']._ObjectId__id.hex()+'.picked.txt','wb'))
    

In [156]:
os.listdir(etri_outcome)

['5a2af415588c1376f0d67afe.txt']

In [153]:
etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/'

In [154]:
filename = rawdata['_id']._ObjectId__id.hex()

pickle.dump(etri[6], open(etri_outcome+rawdata['_id']._ObjectId__id.hex()+'.txt','wb'))

In [92]:
rawdata

{'_id': ObjectId('5a2af415588c1376f0d67afe'),
 'category': '뉴스',
 'date': '2017.12.06',
 'keywords': ['당뇨병', '탄수화물', '식이섬유'],
 'link': 'http://v.media.daum.net/v/20171206100855949',
 'mainText': " 사람들마다 식사를 할 때 밥과 반찬을 먹는 순서가 다르다. 밥부터 먼저 먹는 사람이 있는가 하면, 반찬을 먼저 먹은 후에 밥을 먹는 식이다. 그런데 식사를 할 때 무엇을 먼저 먹는지에 따라 살이 빠지고, 당뇨병까지 예방할 수 있다. 방법은 쉽다. 본인의 한 끼 식사를 영양소별로 나눠 먹는 순서를 정하고, 정한 순서대로 섭취해 살을 빼는 다이어트법이다. 우선 식단 내 영양소를 식이섬유, 단백질, 탄수화물, 크게 세 가지로 나눈다. 식이섬유가 많은 식품에는 채소·과일, 단백질이 많은 식품에는 고기·생선, 탄수화물이 많은 식품에는 쌀·보리 같은 곡류가 있다. 이때 순서를 식이섬유→단백질→탄수화물 순으로 음식을 섭취하면 된다. 예를 들어 일반적인 한식을 먹는다고 하면 나물과 김치를 먼저 먹은 후 다음에 국이나 찌개를 먹는다. 그런 후에 육류나 생선류를 먹고, 마지막으로 밥을 먹는다. 먹는 순서를 식이섬유→단백질→탄수화물 순으로 바꾸면 혈당이 급격하게 상승하는 것을 막고, 빠른 포만감을 준다. 또한 식이섬유부터 먹으면, 단백질·탄수화물이 천천히 흡수돼 혈당이 크게 상승하지 못하게 한다. 혈당이 높아지면 모두 에너지로 쓰이지 못하고 남으면서 지방으로 바뀌고, 비만으로 이어진다. 또 식이섬유를 먼저 먹으면 포만감이 잘 들어, 이후 먹는 단백질과 탄수화물 섭취량이 줄어든다. 특히 혈당을 많이 높이는 탄수화물 섭취를 막아 효과적이다. 실제로 쌀밥을 먹기 전에 생선이나 육류를 먼저 섭취하면 혈당이 급격히 높아지는 것을 억제할 수 있다는 연구결과도 있다. 간사이전력 의학연구소의 야베 다이스케 부소장 등 연구팀은 제2형 당뇨병 환자 1

In [145]:
x = rawdata['_id']
x

ObjectId('5a2af415588c1376f0d67afe')

In [146]:
rawdata['_id']._ObjectId__id.hex()

'5a2af415588c1376f0d67afe'

In [106]:
vect4 = TfidfVectorizer(tokenizer=ct.nouns, ngram_range=(1,4),
                    stop_words=stopwords)
vect4.fit(x1)
#vect4.fit(x2)
#vect4.fit(x3)
#vect4.fit(x4)
#vect4.fit(x5)
count4 = vect4.transform(re.sub('\. ','.\n', rawdata['mainText']).split('\n')).toarray().sum(axis = 0)
idx4 = np.argsort(-count4)
count4 = count4[idx4]
feature_name4 = np.array(vect4.get_feature_names())[idx4]
out4 = dict(zip(feature_name4, count4))
sorted(out4.items(), key=lambda x: x[1], reverse=True)

[('먼저', 3.1991445890560875),
 ('탄수화물', 2.6543833270054069),
 ('혈당', 2.1660505751638253),
 ('섭취', 2.0656146575606704),
 ('식사', 1.9807014416068316),
 ('밥', 1.8165329867679629),
 ('순서', 1.4554532208612323),
 ('때', 1.3101122459112935),
 ('육류', 1.2925200347613497),
 ('결과', 1.1068065846000599),
 ('반찬', 1.0941912368203977),
 ('음식', 1.0550180680448702),
 ('생선', 1.0275869795587738),
 ('방법', 1.0),
 ('약', 0.922033192262804),
 ('포만', 0.91666074702894129),
 ('쪽', 0.82471343292651123),
 ('모두', 0.82245878295543684),
 ('쌀밥', 0.81505517859491472),
 ('못하', 0.80727771423877104),
 ('상승', 0.79691245868892668),
 ('이후', 0.7686047624863166),
 ('지방', 0.74692960827456634),
 ('흡수', 0.74021466432853034),
 ('수', 0.7355702248518694),
 ('법', 0.72510846817730723),
 ('식품', 0.7002158190954183),
 ('거꾸로', 0.65147782389149533),
 ('많이', 0.63374184535807143),
 ('쌀', 0.58993722319706565),
 ('억제', 0.56217834627829988),
 ('그룹', 0.5481970489143585),
 ('그런', 0.54048114891124832),
 ('마지막', 0.54048114891124832),
 ('보다', 0.54032471

In [102]:
vect4 = TfidfVectorizer(tokenizer=ct.nouns, ngram_range=(1,4),
                    stop_words=stopwords)
vect4.fit(x1)
vect4.fit(x2)
vect4.fit(x3)
vect4.fit(x4)
vect4.fit(x5)
count4 = vect4.transform(re.sub('\. ','.\n', rawdata['mainText']).split('\n')).toarray().sum(axis = 0)
idx4 = np.argsort(-count4)
count4 = count4[idx4]
feature_name4 = np.array(vect4.get_feature_names())[idx4]
out4 = dict(zip(feature_name4, count4))
sorted(out4.items(), key=lambda x: x[1], reverse=True)

[('먼저', 2.647840006657419),
 ('탄수화물', 2.1459401777922396),
 ('혈당', 1.9388114653496136),
 ('섭취', 1.8544457171625832),
 ('식이섬유', 1.7589750770325798),
 ('식사', 1.7216141555297726),
 ('밥', 1.6599826826000881),
 ('단백질 탄수화물', 1.6486512379756904),
 ('단백질', 1.5430532660345244),
 ('때', 1.1868717714773676),
 ('육류', 1.136065104807138),
 ('생선', 1.0704668272380691),
 ('반찬', 1.0386737408354931),
 ('결과', 1.0359573708415231),
 ('방법', 1.0),
 ('순서', 0.98891256179281828),
 ('식이섬유 단백질', 0.96231817371702055),
 ('식이섬유 단백질 탄수화물', 0.96231817371702055),
 ('약', 0.90940513474627638),
 ('음식', 0.89823743937253098),
 ('모두', 0.82078470549335925),
 ('당뇨병', 0.7917063814422356),
 ('많이', 0.78857451588652427),
 ('탄수화물 섭취', 0.77492111968885835),
 ('하면', 0.75217301280355808),
 ('지방', 0.70692934631864146),
 ('못하', 0.69871967978438132),
 ('흡수', 0.68864377975647906),
 ('밥 반찬', 0.6818659951352265),
 ('크게', 0.67329333214650322),
 ('식품', 0.66834153043089328),
 ('영양소', 0.63232628428393345),
 ('쌀밥', 0.62360273900643592),
 ('법', 0.6

In [103]:
vect4 = TfidfVectorizer(tokenizer=ct.phrases, ngram_range=(1,1),
                    stop_words=stopwords)
vect4.fit(x1)
vect4.fit(x2)
vect4.fit(x3)
vect4.fit(x4)
vect4.fit(x5)
count4 = vect4.transform(re.sub('\. ','.\n', rawdata['mainText']).split('\n')).toarray().sum(axis = 0)
idx4 = np.argsort(-count4)
count4 = count4[idx4]
feature_name4 = np.array(vect4.get_feature_names())[idx4]
out4 = dict(zip(feature_name4, count4))
sorted(out4.items(), key=lambda x: x[1], reverse=True)

[('탄수화물', 2.652335239011482),
 ('먼저', 2.4179824299316746),
 ('혈당', 2.3602571608616172),
 ('식이섬유', 2.2175272857255952),
 ('단백질', 2.2175272857255952),
 ('식사', 1.8872972053190649),
 ('생선', 1.8636578729641378),
 ('순서', 1.6025836028749274),
 ('육류', 1.5038089788191193),
 ('반찬', 1.3741627744056613),
 ('결과', 1.2912052314802238),
 ('음식', 1.0897177618204064),
 ('모두', 1.0303623867958787),
 ('사람', 1.0177645503207056),
 ('당뇨병', 1.0013885040260471),
 ('방법', 1.0),
 ('흡수', 0.88096878833416103),
 ('크게', 0.85025408269105274),
 ('탄수화물 순', 0.84490825272125814),
 ('지방', 0.83906911620484981),
 ('영양소', 0.83383561820569918),
 ('쌀밥', 0.76189358774877203),
 ('다이어트', 0.7081738331720937),
 ('억제', 0.69873344549256999),
 ('포만', 0.69602909827030424),
 ('포만감', 0.69602909827030424),
 ('연구', 0.6588459372987524),
 ('이후', 0.65877316918563777),
 ('과일', 0.64192227117676137),
 ('분비', 0.6282627118996873),
 ('30', 0.62790748288999576),
 ('거꾸로 식사법', 0.62286355415526962),
 ('무엇', 0.59548008297116284),
 ('거꾸로', 0.588302165821138

In [113]:
vect4 = TfidfVectorizer(tokenizer=ct.phrases, ngram_range=(1,1),
                    stop_words=stopwords)
vect4.fit(x1)
vect4.fit(x2)
vect4.fit(x3)
vect4.fit(x4)
vect4.fit(x5)
count4 = vect4.transform(re.sub('\. ','.\n', rawdata['mainText']).split('\n')).toarray().sum(axis = 0)
idx4 = np.argsort(-count4)
count4 = count4[idx4]
feature_name4 = np.array(vect4.get_feature_names())[idx4]
out4 = dict(zip(feature_name4, count4))
sorted(out4.items(), key=lambda x: x[1], reverse=True)

[('탄수화물', 2.652335239011482),
 ('먼저', 2.4179824299316746),
 ('혈당', 2.3602571608616172),
 ('식이섬유', 2.2175272857255952),
 ('단백질', 2.2175272857255952),
 ('식사', 1.8872972053190649),
 ('생선', 1.8636578729641378),
 ('순서', 1.6025836028749274),
 ('육류', 1.5038089788191193),
 ('반찬', 1.3741627744056613),
 ('결과', 1.2912052314802238),
 ('음식', 1.0897177618204064),
 ('모두', 1.0303623867958787),
 ('사람', 1.0177645503207056),
 ('당뇨병', 1.0013885040260471),
 ('방법', 1.0),
 ('흡수', 0.88096878833416103),
 ('크게', 0.85025408269105274),
 ('탄수화물 순', 0.84490825272125814),
 ('지방', 0.83906911620484981),
 ('영양소', 0.83383561820569918),
 ('쌀밥', 0.76189358774877203),
 ('다이어트', 0.7081738331720937),
 ('억제', 0.69873344549256999),
 ('포만', 0.69602909827030424),
 ('포만감', 0.69602909827030424),
 ('연구', 0.6588459372987524),
 ('이후', 0.65877316918563777),
 ('과일', 0.64192227117676137),
 ('분비', 0.6282627118996873),
 ('30', 0.62790748288999576),
 ('거꾸로 식사법', 0.62286355415526962),
 ('무엇', 0.59548008297116284),
 ('거꾸로', 0.588302165821138

In [73]:
Counter(etri[1]).most_common(n=10)

[(('을', 'JKO'), 29),
 (('먹', 'VV'), 28),
 (('.', 'SF'), 25),
 (('를', 'JKO'), 22),
 (('이', 'JKS'), 20),
 (('는', 'ETM'), 18),
 ((',', 'SP'), 17),
 (('다', 'EF'), 16),
 (('먼저', 'MAG'), 15),
 (('ㄴ', 'ETM'), 14)]

In [75]:
Counter(etri[2]).most_common(n=10)

[(('먼저', ''), 10),
 (('먹는', ''), 9),
 (('먹은', ''), 9),
 (('있다.', ''), 5),
 (("먼저'", ''), 5),
 (('식사를', ''), 4),
 (('혈당이', ''), 4),
 (('약', ''), 4),
 (('후에', ''), 3),
 (('순서를', ''), 3)]

In [78]:
Counter(etri[3]).most_common(n=10)

[(('탄수화물', 'MT_CHEMICAL'), 8),
 (('단백질', 'MT_CHEMICAL'), 7),
 (('생선', 'CV_FOOD'), 6),
 (('밥', 'CV_FOOD'), 5),
 (('육류', 'CV_FOOD'), 4),
 (('쌀밥', 'CV_FOOD'), 4),
 (('당뇨병', 'TMM_DISEASE'), 3),
 (('쌀', 'CV_FOOD'), 2),
 (('인슐린', 'MT_CHEMICAL'), 2),
 (('한 끼', 'QT_COUNT'), 1)]

In [79]:
Counter(etri[4]).most_common(n=10)

[(('먼저', 'ARGM-ADV'), 10),
 (('식품', 'ARG2'), 6),
 (('혈당', 'ARG1'), 6),
 (('식이섬유', 'ARG1'), 5),
 (("먼저'", 'ARGM-TMP'), 5),
 (('식사', 'ARG1'), 3),
 (('순서', 'ARG1'), 3),
 (('밥', 'ARG1'), 3),
 (('것', 'ARG1'), 3),
 (('쌀밥', 'ARG1'), 3)]

In [ ]:
def Extract_Keywords(text, stopwords):
    etri = Run_ETRI_Analysis(stopwords, text)
    soy1 = RunLRNounExtractor(text)
    soy2 = RunWordExtractor(text)
    et0 = list(filter(lambda x: x[1] in ['NNG', 'NNP','NNG','SL','SH','NH'], etri[0]))
    et0 = list(map(lambda x: x[0], et0))
    et1 = list(filter(lambda x: x[1] in ['NNG', 'NNP','NNG','SL','SH','NH'], etri[1]))
    et1 = list(map(lambda x: x[0], et1))
    et2 = list(map(lambda x: x[0], etri[2]))
    et3 = list(map(lambda x: x[0], etri[3]))
    et4 = list(filter(lambda x: x[1] in ['ARG0','ARG1','ARG2','ARG3','ARG4'], etri[4]))
    et4 = list(map(lambda x: x[0], et4))
    et5 = list(filter(lambda x: x[1] in ['NP','NP_SBJ','NP_OBJ','NP_AJT','VNP'], etri[5]))
    et5 = list(map(lambda x: x[0], et5))
    mecabout = list(filter(lambda x: x[1] in ['NNG','NNB','NNP'], mecab.pos(text)))
    ctout = list(filter(lambda x: x[1] == 'Noun' , ct.pos(text)))
    otout = list(filter(lambda x: x[1] == 'Noun' , ot.pos(text)))
    mcout = list(map(lambda x: x[0], mecabout))
    ctout = list(map(lambda x: x[0], ctout))
    otout = list(map(lambda x: x[0], otout))
    out = list(soy1.keys())+list(soy2.keys())+ctout+otout+mcout+et3+et4+et0+et1+et2+et5
    vect = TfidfVectorizer().fit(out)
    y = [list(soy1.keys()), list(soy2.keys()),et3, et4, ctout, otout, mcout,et0, et1, et2, et5]
    y = list(filter(lambda x: len(x) !=0, y))
    outdict = dict()
    for i in y:
        count = vect.transform(i).toarray().sum(axis = 0)
        idx = np.argsort(-count)
        count = count[idx]
        feature_name = np.array(vect.get_feature_names())[idx]
        out = dict(zip(feature_name, count))
        for ii in out:
            if not ii in outdict:
                outdict[ii] = out[ii]
            else:
                outdict[ii] +=out[ii]
    x = sorted(outdict.items(), key = itemgetter(1), reverse=True)[:5]
    output = list(map(lambda x: x[0], x))
    return output


In [ ]:
set(map(lambda x: x[0], f)) & set(out.keys())

In [108]:
from gensim.summarization import keywords
#keywords(rawdata['mainText'])


In [109]:
keywords(rawdata['mainText'])

'탄수화물\n혈당이\n먹으면\n단백질\n식사를\n거꾸로\n쌀밥을\n순으로 음식을\n것으로\n나눈다\n그룹보다\n결과를\n생선을\n당뇨병까지\n다르다\n환자나\n한식을\n식이섬유\n글루카곤은 억제하는\n국이나 찌개를\n인르레틴\n관여하는\n걸리는\n흡수될\n비슷한\n코넬대'

In [110]:
from __future__ import print_function
from textrankr import TextRank

textrank = TextRank(rawdata['mainText'])
print(textrank.summarize())

이때 순서를 식이섬유→단백질→탄수화물 순으로 음식을 섭취하면 된다.
먹는 순서를 식이섬유→단백질→탄수화물 순으로 바꾸면 혈당이 급격하게 상승하는 것을 막고, 빠른 포만감을 준다.
거꾸로 식사법은 후식→밥·반찬 순으로 식사를 하는 것인데, 역시 식이섬유를 먼저 섭취하고 이후 단백질·탄수화물을 섭취한다는 공통점이 있다.


In [ ]:
for data in useCollection.find({'site':'daum'}):
    if len(data['mainText'].strip()) !=0:
        if data['keywords'] !='NaN':
            ct.add_dictionary(list(map(lambda x: x.split(' · ')[-1], data['keywords'])), 'Noun')
        y = Extract_Keywords2(data['mainText'],stopwords)
        z = TextRank(data['mainText'])
        p = RunLRNounExtractor(data['mainText'])
        q = RunWordExtractor(data['mainText'])
        #print (y)
        print (data['keywords'])
        print (z.keywords())
        #print (p.keys())
        #print (q.keys())
        print ()
        break

In [111]:
from lexrankr import LexRank

In [112]:
lexrank = LexRank()

In [ ]:
m.cohesion_backward, m.cohesion_forward, m.

# sentiment analysis

In [ ]:
import os
import sys

In [ ]:
def read_data(filename):
    with codecs.open(filename, encoding = 'utf-8', mode='r') as f:
        data